In [1]:
import numpy as np

np.convolve((1, 2, 3), (4, 5, 6))

array([ 4, 13, 28, 27, 18])

In [2]:
arr1 = np.random.random(100000)
arr2 = np.random.random(100000)

In [3]:
%%timeit
np.convolve(arr1, arr2)

937 ms ± 132 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
import scipy.signal 

In [5]:
%%timeit
scipy.signal.fftconvolve(arr1, arr2)

7.88 ms ± 83.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader 
import torch.nn.functional as F

In [31]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5, stride=1)

        self.fc1 = nn.Linear(4000, 50)
        self.fc2 = nn.Linear(50,10)

        self.flatten = nn.Flatten()

    def forward(self, x):
        #print("연산 전", x.size())
        x = F.relu(self.conv1(x))
        #print("conv1 연산 후", x.size())

        x = F.relu(self.conv2(x))
        #print("conv2 연산 후", x.size()) 

        x = self.flatten(x)
        #print("차원 감소 후", x.size())

        x = F.relu(self.fc1(x))
        #print("fc1 연산 후", x.size())

        x = self.fc2(x)
        #print("fc2 연산 후", x.size())

        return x      

In [32]:
cnn = CNN()
output = cnn(torch.randn(10, 1, 28, 28))

In [33]:
import torchvision.transforms as transforms
from torchvision.datasets import MNIST

In [34]:
mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5, ), (1.0, ))
])

In [35]:
download_root = "./MNIST_DATASET"

train_dataset = MNIST(download_root, transform =mnist_transform, train=True, download=True)
test_dataset = MNIST(download_root, transform =mnist_transform, train=False, download=True)

In [36]:
batch_size = 64

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device )

model = CNN().to(device)

cuda


In [38]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [39]:
len(train_loader)

938

In [40]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

In [41]:
model.train()

for epoch in range(epochs):
    avg_loss = 0
    for i, (img_data, label_data) in enumerate(train_loader):
        img_data, label_data = img_data.to(device), label_data.to(device)

        pred = model(img_data)

        loss = loss_function(pred, label_data)
        avg_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss /= len(train_loader)
    print(f"Epoch: [{epoch}/{epochs}] Loss: {avg_loss}")

Epoch: [0/5] Loss: 0.22831517457962036
Epoch: [1/5] Loss: 0.07873327285051346
Epoch: [2/5] Loss: 0.055560093373060226
Epoch: [3/5] Loss: 0.04382827877998352
Epoch: [4/5] Loss: 0.034306298941373825


In [46]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            correct += (pred.argmax(1)==y).type(torch.float).sum().item()

        test_loss /= num_batches
        correct /= size 
        print(f"Test Error: \nAccuracy: {(100*correct):>0.1f}%, Avg Loss: {test_loss:>8f} \n")

In [47]:
test_loop(test_loader, model, loss_function)


Test Error: 
Accuracy: 98.4%, Avg Loss: 0.049440 



In [82]:
class CNNv2(nn.Module):
    def __init__(self):
        super().__init__()
        self.keep_prob = 0.5

        # L1 ImgIN shape = (?, 28, 28, 1)
        #    Conv       -> (?, 28, 28, 32)
        #    Pool       -> (?, 14, 14, 32)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )

        # L4 Fc 4*4*128 inputs -> 623 outputs
        self.fc1 = nn.Linear(4*4*128, 625, bias=True)
        nn.init.xavier_uniform_(self.fc1.weight)

        self.layer4 = nn.Sequential(
            self.fc1, 
            nn.ReLU(),
            nn.Dropout(p=1-self.keep_prob)
        )

        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = nn.Linear(625, 10, bias=True)
        nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [83]:
model = CNNv2().to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [84]:
total_batch = len(train_loader)

In [85]:
for epoch in range(epochs):
    avg_cost = 0

    for X, Y in train_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.171949297
[Epoch:    2] cost = 0.0464844778
[Epoch:    3] cost = 0.0343767591
[Epoch:    4] cost = 0.0261751544
[Epoch:    5] cost = 0.0221783482


In [86]:
test_loop(test_loader, model, criterion)

Test Error: 
Accuracy: 99.3%, Avg Loss: 0.025699 



# 더 깊은 Convolution Network 의 정확도가 더 높은 것을 확인할 수 있었다. 

**깊은 컨볼루션 네트워크**

Test Error: 
Accuracy: 99.3%, Avg Loss: 0.025699 

**간단한 컨볼루션 네트워크**

Test Error: 
Accuracy: 98.4%, Avg Loss: 0.049440 

깊고 효율적으로 네트워크를 쌓는다면 간단한 네트워크보다 월등한 성능을 낼 수 있다.